# Scikit-learn Pre-processing

In [1]:
import sklearn
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

from sklearn import set_config

#Sets output of transform and fit_transform to pandas dataframe output.
set_config(transform_output = "pandas")

## Scaling

In [2]:
tiny_data = np.array([[ 1., -1.,  2.],
                     [ 2.,  0.,  0.],
                     [ 0.,  1., -1.]])

scaler = StandardScaler().fit(tiny_data)
scaler

StandardScaler()

In [3]:
scaler.mean_

array([1.        , 0.        , 0.33333333])

In [4]:
scaler.scale_

array([0.81649658, 0.81649658, 1.24721913])

In [5]:
X_scaled = scaler.transform(tiny_data)

In [6]:
X_scaled.mean()

x0    0.0
x1    0.0
x2    0.0
dtype: float64

In [7]:
X_scaled.std()

x0    1.224745
x1    1.224745
x2    1.224745
dtype: float64

## Ordinal Encoding

In [8]:
# example of a ordinal encoding
from numpy import asarray

In [9]:
# define data
data = asarray([['data'], ['wrangling'], ['rocks']])
print(data)

[['data']
 ['wrangling']
 ['rocks']]


In [10]:
# define ordinal encoding
encoder = OrdinalEncoder()
# transform data
encoder.fit_transform(data)

,x0
0,0.0
1,2.0
2,1.0


## One Hot Encoding

In [11]:
# define one hot encoding
encoder = OneHotEncoder(sparse_output=False)
# transform data
encoder.fit_transform(data)

,x0_data,x0_rocks,x0_wrangling
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0


## Imputing missing values

In [12]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True, parser='auto')

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [13]:
X.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [14]:
y_train.head()

463     0
1190    1
231     1
550     1
1047    1
Name: survived, dtype: category
Categories (2, object): ['0', '1']

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   pclass     1309 non-null   int64   
 1   name       1309 non-null   object  
 2   sex        1309 non-null   category
 3   age        1046 non-null   float64 
 4   sibsp      1309 non-null   int64   
 5   parch      1309 non-null   int64   
 6   ticket     1309 non-null   object  
 7   fare       1308 non-null   float64 
 8   cabin      295 non-null    object  
 9   embarked   1307 non-null   category
 10  boat       486 non-null    object  
 11  body       121 non-null    float64 
 12  home.dest  745 non-null    object  
dtypes: category(2), float64(3), int64(3), object(5)
memory usage: 115.4+ KB


In [16]:
missing = X_test.isnull().sum()
missing = missing[missing > 0].sort_values(ascending = False)

In [17]:
missing

body         302
cabin        249
boat         204
home.dest    135
age           61
embarked       1
dtype: int64

In [18]:
simple_imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
simple_imputed = simple_imp.fit_transform(X_test[['age', 'body']])

In [19]:
#Replace the age and body columns in the original X_Test dataframe
#with the imputed values
X_test[['age', 'body']] = simple_imputed

#Repeat the above code to get number of NA values- note how the 'age' and 
#'body' columns disappear
missing = X_test.isnull().sum()
missing = missing[missing > 0].sort_values(ascending = False)
missing

cabin        249
boat         204
home.dest    135
embarked       1
dtype: int64

## Putting it all together

In [20]:
X_train.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
463,2,"Jefferys, Mr. Ernest Wilfred",male,22.0,2,0,C.A. 31029,31.500,NaN,S,NaN,NaN,"Guernsey / Elizabeth, NJ"
1190,3,"Sap, Mr. Julius",male,25.0,0,0,345768,9.500,NaN,S,11,NaN,NaN
231,1,"Peuchen, Major. Arthur Godfrey",male,52.0,0,0,113786,30.500,C104,S,6,NaN,"Toronto, ON"
550,2,"Richards, Mrs. Sidney (Emily Hocking)",female,24.0,2,3,29106,18.750,NaN,S,4,NaN,"Cornwall / Akron, OH"
1047,3,"Najib, Miss. Adele Kiamie 'Jane'",female,15.0,0,0,2667,7.225,NaN,C,C,NaN,NaN


In [21]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression

# Here we use `StandardScaler` for continuous variables; 
# then we impute for missing data (check the documentation for the imputation method)
# We use `OneHotEncoder` for categorical variables
# NOTE: we are using a subset of the features (not all the columns)

ct = make_column_transformer((make_pipeline(SimpleImputer(),
                                            StandardScaler()), ["age", "fare"]),
                             (OneHotEncoder(sparse_output=False), ["embarked", "sex", "pclass"]), 
                             verbose_feature_names_out=False)

# Note: click on pipeline elements to see more details
clf = make_pipeline(ct, LogisticRegression())
clf

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['embarked', 'sex',
                                                   'pclass'])],
                                   verbose_feature_names_out=False)),
                ('logisticregression', LogisticRegression())])

In [22]:
clf.fit(X_train, y_train)
clf.score(X_train, y_train)

0.7757390417940877

In [23]:
# Let's remove the last step in the pipeline (which is LogisticRegression()) & transform the X_test data
clf[:-1].transform(X_test)

,age,fare,embarked_C,embarked_Q,embarked_S,embarked_nan,sex_female,sex_male,pclass_1,pclass_2,pclass_3
116,2.325291,4.841611,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
449,1.861641,-0.180671,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
598,-0.302057,-0.379470,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
381,0.007043,-0.389933,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
78,2.634390,1.078212,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
11,-0.920256,4.099255,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1303,-0.008393,-0.359416,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
483,-0.008393,0.028591,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1069,-0.008393,-0.337618,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
